In [ ]:
from time import time
# simple stopwatch to time whatevs, in (float) seconds
# keeps track of laps along with final time
class Stopwatch:
    def __init__(self):
            self.start_time = time()
            self.last_time = self.start_time
            self.laps = []
    def lap(self):
        this_time = time()
        delta_time = this_time - self.last_time
        self.laps.append(delta_time)
        self.last_time = this_time
        return delta_time
    def stop(self):
        self.stop_time = time()
        self.delta_time = self.stop_time - self.start_time
        return self.delta_time

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import math
import random
import pickle

# set plot params
plt.rcParams.update({'font.size': 22})
plt.show()
plot_dir = 'Plots/'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

# uci primary colors
uci_blue = (0/255, 62/255, 120/255)
uci_gold = (253/255, 185/255, 19/255)

# uci secondary color palette
uci_light_blue = (106/255, 162/255, 184/255)
uci_light_gray = (197/255, 190/255, 181/255)
uci_dark_blue = (27/255, 61/255, 109/255)
uci_orange = (247/255, 141/255, 45/255)
uci_light_yellow = (247/255, 235/255, 95/255)
uci_dark_gray = (85/255, 87/255, 89/255)
uci_lime_green = (122/255, 184/255, 0/255)

# color blind friendly colors
# https://gist.github.com/thriveth/8560036
color_blinds = {
    'blue':   [55/255,  126/255, 184/255],  #377eb8 
    'orange': [255/255, 127/255, 0/255],    #ff7f00
    'green':  [77/255,  175/255, 74/255],   #4daf4a
    'pink':   [247/255, 129/255, 191/255],  #f781bf
    'brown':  [166/255, 86/255,  40/255],   #a65628
    'purple': [152/255, 78/255,  163/255],  #984ea3
    'gray':   [153/255, 153/255, 153/255],  #999999
    'red':    [228/255, 26/255,  28/255],   #e41a1c
    'yellow': [222/255, 222/255, 0/255]     #dede00
} 
color_blinds_list = [color_blinds[color] for color in color_blinds]

object_color = uci_blue

In [ ]:
import binvox as bv
binvox_path = 'map_voxels.binvox'
voxels = bv.Binvox.read(binvox_path, 'dense')
voxels_data = voxels.data
voxels_scale = voxels.scale 
voxels_trans = voxels.translate
voxels_res = (np.absolute(voxels_trans)) * 2 * voxels_scale

In [ ]:
_global_plt_patches = None
map_2d = None
map_3d = None
pad_size = 3
floor_dim = 0

x_range = [-150, 150]
nX = x_range[1] - x_range[0]
y_range = [-125, 100]
nY = y_range[1] - y_range[0]
z_range = [0, 50]
nZ = z_range[1] - z_range[0]
def xi_x(xi):
    return xi + x_range[0]
def yi_y(yi):
    return yi + y_range[0]
def zi_z(zi):
    return zi + z_range[0]

def plot_map(axis, object_color='black'):
    global _global_plt_patches
    global map_2d
    global map_3d
    global floor_dim
    if _global_plt_patches is None:
        map_2d = np.zeros((nX, nY), dtype=int)
        map_3d = np.zeros((nX, nY, nZ), dtype=int)
        origin = voxels_data[int(voxels_data.shape[1]/2), int(voxels_data.shape[0]/2), :]
        floor_dim = max([i for i, x in enumerate(origin) if x])
        scale = voxels_scale * 100
        shift = voxels_trans
        _global_plt_patches = []
        for x in range(voxels_data.shape[0]):
            for y in range(voxels_data.shape[1]):
                for z in range(voxels_data.shape[2]-1,-1,-1):
                    if voxels_data[x, y, z] and z > floor_dim:
                        #x_loc = x - shift - scale/2 # align to left for plt.rect
                        #y_loc = y - shift - scale/2 # align to bottom for plt.rect
                        #patch = patches.Rectangle((x_loc, y_loc), scale, scale, color = object_color)
                        _x = x + shift[0]
                        _y = y + shift[1]
                        _z = z + shift[2]
                        patch = patches.Rectangle((_x, _y), 1, 1, color = object_color)
                        _global_plt_patches.append(patch)
                        x_i = int(_x - x_range[0])
                        y_i = int(_y - y_range[0])
                        z_i = int(_z - z_range[0])
                        map_2d[x_i,y_i] = 1
                        for i in range(-1*pad_size,1+pad_size):
                            for j in range(-1*pad_size,1+pad_size):
                                for k in range(z_i+4, -1, -1):
                                    if k >= nZ:
                                        continue
                                    map_3d[x_i+i,y_i+j,k] = 1
                        break
        map_2d_padded = map_2d.copy()
        for x in range(map_2d.shape[0]):
            for y in range(map_2d.shape[1]):
                if map_2d[x,y] == 1:
                    for i in range(-1*pad_size,1+pad_size):
                        for j in range(-1*pad_size,1+pad_size):
                            if x+i >= nX:
                                continue
                            if y+j >= nY:
                                continue
                            if map_2d[x+i,y+j] == 0:
                                patch = patches.Rectangle((x+i+x_range[0], y+j+y_range[0]), 1, 1, color = 'red')
                                _global_plt_patches.append(patch)
                                map_2d_padded[x+i,y+j] = 1
        map_2d = map_2d_padded.copy()
        floor_dim = floor_dim + shift[2]- z_range[0] 
    # add list of patches (much quicker than iteratively drawing)
    map_stuff = PatchCollection(_global_plt_patches, match_original=True)
    # axis.gca().add_collection(map_stuff)
    axis.add_collection(map_stuff)

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1)
#fig.set_size_inches(8, 8)
# PLOT DRONE PATH
ax1.set_title(f'Drone Path (to scale)')#' Epoch #{set_num}')
ax1.set_xlabel('y [meters]')
ax1.set_ylabel('x [meters]')
ax1.set_xlim(x_range[0], x_range[1])
ax1.set_ylim(y_range[0], y_range[1])
# show objects on map from binvox
plot_map(ax1)
plt.show()

In [ ]:
def get_roof(x, y):
    for z in range(nZ-1,-1,-1):
        if map_3d[x, y, z] > 0:
            return z
    return floor_dim + 4
roofs = np.zeros((nX, nY), dtype=int)
for x in range(nX):
    for y in range(nY):
        roofs[x, y] = get_roof(x,y)
plt.imshow(roofs, cmap='hot', interpolation='nearest')
plt.show()
print(np.max(roofs))

In [ ]:
train_X = (-150, 150-1)
train_Y = (0, 100-1)
val_X = (-150, 0-1)
val_Y = (-125, 0-1)
test_X = (0, 150-1)
test_Y = (-125, 0-1)

In [ ]:
# horizontal motion only
class Node:
    """
        A node class for A* Pathfinding
        parent is parent of the current Node
        position is current position of the Node in the maze
        g is cost from start to current Node
        h is heuristic based estimated cost for current Node to end Node
        f is total cost of present node i.e. :  f = g + h
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position

#This function return the path of the search
def return_path(current_node,maze):
    path = []
    no_rows, no_columns = np.shape(maze)
    # here we create the initialized result maze with -1 in every position
    #result = [[-1 for i in range(no_columns)] for j in range(no_rows)]
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    path = path[::-1]
    '''
    # Return reversed path as we need to show from start to end path
    path = path[::-1]
    start_value = 0
    # we update the path of start to end found by A-star serch with every step incremented by 1
    for i in range(len(path)):
        result[path[i][0]][path[i][1]] = start_value
        start_value += 1
    '''
    return path

move  =  np.array([
    [-1, 0, 1], # go up
          [ 0, -1, 1], # go left
          [ 1, 0, 1 ], # go down
          [ 0, 1, 1 ], # go right
          [ -1, 1, 1 ], # diag right up
          [ 1, 1, 1 ], # diag right down
          [ -1, -1, 1 ], # diag left up
          [ 1, -1, 1 ], # diag left down
],dtype=int)
scaled_moves = []
for scale in [2,4,6,8]:
    for m in move:
        scaled_move = m.copy()
        m[2] = scale
        scaled_moves.append(scaled_move)
move = scaled_moves + list(move)


def search(maze, cost, start, end):
    """
        Returns a list of tuples as a path from the given start to the given end in the given maze
        :param maze:
        :param cost
        :param start:
        :param end:
        :return:
    """

    # Create start and end node with initized values for g, h and f
    start_node = Node(None, tuple(start))
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, tuple(end))
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both yet_to_visit and visited list
    # in this list we will put all node that are yet_to_visit for exploration. 
    # From here we will find the lowest cost node to expand next
    yet_to_visit_list = []  
    # in this list we will put all node those already explored so that we don't explore it again
    visited_list = [] 
    
    # Add the start node
    yet_to_visit_list.append(start_node)
    
    # Adding a stop condition. This is to avoid any infinite loop and stop 
    # execution after some reasonable number of steps
    outer_iterations = 0
    max_iterations = (len(maze) // 2) ** 10

    # what squares do we search . serarch movement is left-right-top-bottom 
    #(4 movements) from every positon

    """
        1) We first get the current node by comparing all f cost and selecting the lowest cost node for further expansion
        2) Check max iteration reached or not . Set a message and stop execution
        3) Remove the selected node from yet_to_visit list and add this node to visited list
        4) Perofmr Goal test and return the path else perform below steps
        5) For selected node find out all children (use move to find children)
            a) get the current postion for the selected node (this becomes parent node for the children)
            b) check if a valid position exist (boundary will make few nodes invalid)
            c) if any node is a wall then ignore that
            d) add to valid children node list for the selected parent
            
            For all the children node
                a) if child in visited list then ignore it and try next node
                b) calculate child node g, h and f values
                c) if child in yet_to_visit list then ignore it
                d) else move the child to yet_to_visit list
    """
    #find maze has got how many rows and columns 
    no_rows, no_columns = np.shape(maze)
    
    # Loop until you find the end
    
    while len(yet_to_visit_list) > 0:
        
        # Every time any node is referred from yet_to_visit list, counter of limit operation incremented
        outer_iterations += 1    

        
        # Get the current node
        current_node = yet_to_visit_list[0]
        current_index = 0
        for index, item in enumerate(yet_to_visit_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
                
        # if we hit this point return the path such as it may be no solution or 
        # computation cost is too high
        if outer_iterations > max_iterations:
            print ("giving up on pathfinding too many iterations")
            return return_path(current_node,maze)

        # Pop current node out off yet_to_visit list, add to visited list
        yet_to_visit_list.pop(current_index)
        visited_list.append(current_node)

        # test if goal is reached or not, if yes then return the path
        if current_node == end_node:
            return return_path(current_node,maze)

        # Generate children from all adjacent squares
        children = []

        for new_position in move: 

            # Get node position
            x = int(current_node.position[0] + new_position[2]*new_position[0])
            y = int(current_node.position[1] + new_position[2]*new_position[1])
            node_position = (x, y)

            # Make sure within range (check if within maze boundary)
            if (node_position[0] > (no_rows - 1) or 
                node_position[0] < 0 or 
                node_position[1] > (no_columns -1) or 
                node_position[1] < 0):
                continue

            # Make sure walkable terrain
            collision = False
            for s in range(1,new_position[2]+1):
                x = int(current_node.position[0] + s*new_position[0])
                y = int(current_node.position[1] + s*new_position[1])
                if maze[x][y] != 0:
                    collision = True
                    break
            if collision:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            
            # Child is on the visited list (search entire visited list)
            if len([visited_child for visited_child in visited_list if visited_child == child]) > 0:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + cost
            ## Heuristic costs calculated here, this is using eucledian distance
            child.h = (((child.position[0] - end_node.position[0]) ** 2) + 
                       ((child.position[1] - end_node.position[1]) ** 2)) 

            child.f = child.g + child.h

            # Child is already in the yet_to_visit list and g cost is already lower
            if len([i for i in yet_to_visit_list if child == i and child.g > i.g]) > 0:
                continue

            # Add the child to the yet_to_visit list
            yet_to_visit_list.append(child)

if __name__ == '__main__':
    '''
    maze = [[0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0]]
    '''
    maze = map_2d.copy()
    
    batches = ['train', 'val', 'test']
    for batch in batches:
        if batch == 'train':
            n = 600
        if batch == 'val':
            n = 200
        if batch == 'test':
            n = 200
        paths = []
        for pI in range(n):
            while True:
                if batch == 'train':
                    _x_range, _y_range = train_X, train_Y
                if batch == 'val':
                    _x_range, _y_range = val_X, val_Y
                if batch == 'test':
                    _x_range, _y_range = test_X, test_Y
                x1 = random.randint(*_x_range) - x_range[0]
                y1 = random.randint(*_y_range) - y_range[0]
                x2 = random.randint(*_x_range) - x_range[0]
                y2 = random.randint(*_y_range) - y_range[0]
                #print(x1, y1, x2, y2)
                if maze[x1][y1] == 0 and maze[x2][y2] == 0:
                    break
            start = [x1, y1] # starting position
            end = [x2, y2] # ending position
            print(batch,start,'to',end,pI,'of',n)
            cost = 1 # cost per movement

            path = search(maze, cost, start, end)

            '''
            fig, ax1 = plt.subplots(nrows=1, ncols=1)
            # PLOT DRONE PATH
            ax1.set_title(f'Drone Path (to scale)')#' Epoch #{set_num}')
            ax1.set_xlabel('y [meters]')
            ax1.set_ylabel('x [meters]')
            ax1.set_xlim(x_range[0], x_range[1])
            ax1.set_ylim(y_range[0], y_range[1])
            # show objects on map from binvox
            plot_map(ax1)
            path_patches = []
            '''
            steps = len(path)
            path_points = [None]*steps
            for step_number, pos in enumerate(path):
                x, y = pos
                _x = x + x_range[0]
                _y = y + y_range[0]
                #patch = patches.Rectangle((_x, _y), 1, 1, color = 'green')
                #path_patches.append(patch)
                point = [_y, _x, -4, 0]
                path_points[step_number] = point.copy()
                #ax1.scatter(x+shift[0], y+shift[1], color='green')
            for i in range(len(path_points)-1):
                this_point = np.array(path_points[i])
                next_point = np.array(path_points[i+1])
                distance_vector = next_point - this_point
                yaw_1_2 = math.atan2(distance_vector[1], distance_vector[0])
                path_points[i][3] = yaw_1_2
                #print(path_points[i], math.degrees(yaw_1_2))
            path_points[-1][3] = yaw_1_2
            #map_stuff = PatchCollection(path_patches, match_original=True)
            #ax1.add_collection(map_stuff)
            #plt.show()
            paths.append(path_points)
            file = open(f'paths_horizontal_{batch}_part0.p', 'wb')
            pickle.dump(paths, file)

In [ ]:
# with vertical motion
class Node:
    """
        A node class for A* Pathfinding
        parent is parent of the current Node
        position is current position of the Node in the maze
        g is cost from start to current Node
        h is heuristic based estimated cost for current Node to end Node
        f is total cost of present node i.e. :  f = g + h
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position

#This function return the path of the search
def return_path(current_node, maze):
    path = []
    no_rows, no_columns, no_zs = np.shape(maze)
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    path = path[::-1]
    return path

move  =  np.array([
    [-1, 0, 0, 1], # go up
          [ 0, -1, 0, 1], # go left
          [ 1, 0, 0, 1 ], # go down
          [ 0, 1, 0, 1 ], # go right
          [ -1, 1, 0, 1 ], # diag right up
          [ 1, 1, 0, 1 ], # diag right down
          [ -1, -1, 0, 1 ], # diag left up
          [ 1, -1, 0, 1 ], # diag left down
    
        [0, 0, 1, 1], # go out
          [ 0, 0, -1, 1], # go in
    
],dtype=int)
scaled_moves = []
for scale in [2,4,6,8]:
    for m in move:
        scaled_move = m.copy()
        m[3] = scale
        scaled_moves.append(scaled_move)
move = scaled_moves + list(move)


def search(maze, cost, start, end):
    """
        Returns a list of tuples as a path from the given start to the given end in the given maze
        :param maze:
        :param cost
        :param start:
        :param end:
        :return:
    """

    # Create start and end node with initized values for g, h and f
    start_node = Node(None, tuple(start))
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, tuple(end))
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both yet_to_visit and visited list
    # in this list we will put all node that are yet_to_visit for exploration. 
    # From here we will find the lowest cost node to expand next
    yet_to_visit_list = []  
    # in this list we will put all node those already explored so that we don't explore it again
    visited_list = [] 
    
    # Add the start node
    yet_to_visit_list.append(start_node)
    
    # Adding a stop condition. This is to avoid any infinite loop and stop 
    # execution after some reasonable number of steps
    outer_iterations = 0
    max_iterations = 10_000 # (len(maze) // 2) ** 10

    # what squares do we search . serarch movement is left-right-top-bottom 
    #(4 movements) from every positon

    """
        1) We first get the current node by comparing all f cost and selecting the lowest cost node for further expansion
        2) Check max iteration reached or not . Set a message and stop execution
        3) Remove the selected node from yet_to_visit list and add this node to visited list
        4) Perofmr Goal test and return the path else perform below steps
        5) For selected node find out all children (use move to find children)
            a) get the current postion for the selected node (this becomes parent node for the children)
            b) check if a valid position exist (boundary will make few nodes invalid)
            c) if any node is a wall then ignore that
            d) add to valid children node list for the selected parent
            
            For all the children node
                a) if child in visited list then ignore it and try next node
                b) calculate child node g, h and f values
                c) if child in yet_to_visit list then ignore it
                d) else move the child to yet_to_visit list
    """
    #find maze has got how many rows and columns 
    no_rows, no_columns, no_zs = np.shape(maze)
    
    # Loop until you find the end
    
    while len(yet_to_visit_list) > 0:
        
        # Every time any node is referred from yet_to_visit list, counter of limit operation incremented
        outer_iterations += 1    

        
        # Get the current node
        current_node = yet_to_visit_list[0]
        current_index = 0
        for index, item in enumerate(yet_to_visit_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
                
        # if we hit this point return the path such as it may be no solution or 
        # computation cost is too high
        if outer_iterations > max_iterations:
            print ("giving up on pathfinding too many iterations")
            return return_path(current_node,maze), outer_iterations

        # Pop current node out off yet_to_visit list, add to visited list
        yet_to_visit_list.pop(current_index)
        visited_list.append(current_node)

        # test if goal is reached or not, if yes then return the path
        if current_node == end_node:
            return return_path(current_node, maze), outer_iterations

        # Generate children from all adjacent squares
        children = []

        for new_position in move: 

            # Get node position
            x = int(current_node.position[0] + new_position[3]*new_position[0])
            y = int(current_node.position[1] + new_position[3]*new_position[1])
            z = int(current_node.position[2] + new_position[3]*new_position[2])
            node_position = (x, y, z)

            # Make sure within range (check if within maze boundary)
            if (node_position[0] > (no_rows - 1) or 
                node_position[0] < 0 or 
                node_position[1] > (no_columns -1) or 
                node_position[1] < 0 or 
                node_position[2] > (no_zs -1) or 
                node_position[2] < 0):
                continue

            # Make sure walkable terrain
            collision = False
            for s in range(1,new_position[3]+1):
                x = int(current_node.position[0] + s*new_position[0])
                y = int(current_node.position[1] + s*new_position[1])
                z = int(current_node.position[2] + s*new_position[2])
                roof = get_roof(x, y)
                if z < roof:
                    collision = True
                    break
            if collision:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            
            # Child is on the visited list (search entire visited list)
            if len([visited_child for visited_child in visited_list if visited_child == child]) > 0:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + cost
            ## Heuristic costs calculated here, this is using eucledian distance
            child.h = (((child.position[0] - end_node.position[0]) ** 2) + 
                       ((child.position[1] - end_node.position[1]) ** 2) + 
                       ((child.position[2] - end_node.position[2]) ** 2)
                      ) 

            child.f = child.g + child.h

            # Child is already in the yet_to_visit list and g cost is already lower
            if len([i for i in yet_to_visit_list if child == i and child.g > i.g]) > 0:
                continue

            # Add the child to the yet_to_visit list
            yet_to_visit_list.append(child)


if __name__ == '__main__':
    '''
    maze = [[0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 0, 0, 1, 0]]
    '''
    maze = map_3d.copy()
    
    batches = ['train', 'val', 'test']
    for batch in batches:
        if batch == 'train':
            n = 600
        if batch == 'val':
            n = 200
        if batch == 'test':
            n = 200
        paths = []
        for pI in range(n):
            if batch == 'train':
                _x_range, _y_range = train_X, train_Y
            if batch == 'val':
                _x_range, _y_range = val_X, val_Y
            if batch == 'test':
                _x_range, _y_range = test_X, test_Y
                
            while True:
                x1 = random.randint(*_x_range) - x_range[0]
                y1 = random.randint(*_y_range) - y_range[0]
                z1 = roofs[x1, y1] + 2

                x2 = random.randint(*_x_range) - x_range[0]
                y2 = random.randint(*_y_range) - y_range[0]
                z2 = roofs[x2, y2] + 2

                linear_distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
                if linear_distance < 100:
                    break
            
            start = [x1, y1, z1] # starting position
            end = [x2, y2, z2] # ending position
            print(batch,start,'to',end,pI,'of',n)
            cost = 1 # cost per movement
            sw = Stopwatch()
            path, outer_iterations = search(maze, cost, start, end)
            dt = sw.stop()

            '''
            fig, ax1 = plt.subplots(nrows=1, ncols=1)
            # PLOT DRONE PATH
            ax1.set_title(f'Drone Path (to scale)')#' Epoch #{set_num}')
            ax1.set_xlabel('y [meters]')
            ax1.set_ylabel('x [meters]')
            ax1.set_xlim(x_range[0], x_range[1])
            ax1.set_ylim(y_range[0], y_range[1])
            # show objects on map from binvox
            plot_map(ax1)
            path_patches = []
            '''
            steps = len(path)
            path_points = [None]*(steps+1)
            path_points[-1] = [dt, outer_iterations]
            for step_number, pos in enumerate(path):
                x, y, z = pos
                _x = x + x_range[0]
                _y = y + y_range[0]
                _z = z + z_range[0]
                #patch = patches.Rectangle((_x, _y), 1, 1, color = 'green')
                #path_patches.append(patch)
                point = [_y, _x, -1*_z, 0]
                path_points[step_number] = point.copy()
                #ax1.scatter(x+shift[0], y+shift[1], color='green')
            for i in range(len(path_points)-2):
                this_point = np.array(path_points[i])
                next_point = np.array(path_points[i+1])
                distance_vector = next_point - this_point
                yaw_1_2 = math.atan2(distance_vector[1], distance_vector[0])
                path_points[i][3] = yaw_1_2
                #print(path_points[i], math.degrees(yaw_1_2))
            path_points[-2][3] = yaw_1_2
            # add list of patches (much quicker than iteratively drawing)
            #map_stuff = PatchCollection(path_patches, match_original=True)
            # axis.gca().add_collection(map_stuff)
            #ax1.add_collection(map_stuff)
            #plt.show()
            paths.append(path_points)
            file = open(f'paths_vertical_{batch}_part0.p', 'wb')
            pickle.dump(paths, file)

In [ ]:
x1, y1, z1 = [174, 106, 20]
x2, y2, z2 = [203, 1, 5]

fig, ax1 = plt.subplots(nrows=1, ncols=1)
#fig.set_size_inches(8, 8)
# PLOT DRONE PATH
ax1.set_title(f'Drone Path (to scale)')#' Epoch #{set_num}')
ax1.set_xlabel('y [meters]')
ax1.set_ylabel('x [meters]')
ax1.set_xlim(x_range[0], x_range[1])
ax1.set_ylim(y_range[0], y_range[1])
# show objects on map from binvox
plot_map(ax1)
plt.scatter(xi_x(x1), yi_y(y1), color='blue')
print(zi_z(z1))
plt.scatter(xi_x(x2), yi_y(y2), color='green')
print(zi_z(z2))
plt.show()

In [ ]:
file = open(f'paths_horizontal_test_part0.p', 'rb')
paths = pickle.load(file)

In [ ]:
paths